In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import pydicom
import os
import sys
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor
import cv2

import tensorflow as tf
import keras

import seaborn as sns

from glob import glob
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.optimizers import Optimizer
from keras.callbacks import *

In [ ]:
#sys.path.append(os.path.abspath('../input/efficientnet/efficientnet-master/efficientnet-master'))

For windowing the input images . Many thanks

https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing
https://www.kaggle.com/akensert/resnet50-keras-baseline-model
https://www.kaggle.com/xhlulu/rsna-intracranial-simple-densenet-in-keras/notebook
https://www.kaggle.com/taindow/pytorch-efficientnet-b0
https://www.kaggle.com/rajnishe/efficenet-b5 *
-- For analysis of data

https://www.kaggle.com/allunia/rsna-ih-detection-eda-baseline
https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial

In [ ]:
# Path Information
# Always clean new files after kernel 
# else you will get too many open file issue

BASE_PATH = '/kaggle/input/rsna_noben_png_224_224_v1/'

#CSV_BASE_PATH = '/kaggle/input/rsna-intracranial-hemorrhage-detection/'

TRAIN_PATH = 'stage_1_train_images/'
TEST_PATH = 'stage_1_test_images/'

TMP_DIR_PATH = '/kaggle/tmp/'

TRAIN_TMP_DIR_PATH = TMP_DIR_PATH + 'train_png_img/'
TEST_TMP_DIR_PATH  = TMP_DIR_PATH + 'test_png_img/'

IMG_SIZE = 224

In [ ]:
train_df_csv = pd.read_csv(BASE_PATH + 'stage_1_train.csv')

print(train_df_csv.shape)
#train_df_csv.head()

In [ ]:
#Extract image name
train_df_csv['filename'] = train_df_csv['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".dcm")

#Extract type of problem
train_df_csv['type'] = train_df_csv['ID'].apply(lambda st: st.split('_')[2])

print(train_df_csv.shape)
#train_df_csv.head(8)

In [ ]:
train_df_csv.filename.nunique()

Analysis
Finding distribution of labels

In [ ]:
#train_df_csv.groupby("type").size()
label_dist = train_df_csv.groupby("type").Label.value_counts().unstack().reset_index()
       
#.Label.value_counts()#.unstack()
label_dist

In [ ]:
#label_dist.columns = ['type','neg_label','pos_label']

#label_dist

In [ ]:
'''
g = sns.catplot(x="type", y="neg_label",
...                 data=label_dist, saturation=.5,
...                 kind="bar", ci=None, aspect=1.8)
g = sns.catplot(x="type", y="pos_label",
...                 data=label_dist, saturation=.5,
...                 kind="bar", ci=None, aspect=1.8)
'''

# Converting train CSV File
-- Ready for processing , model creation

In [ ]:
# Pivot it so that one row contains one image and all labels
pivot_train_df_csv = train_df_csv[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()
print(pivot_train_df_csv.shape)
pivot_train_df_csv.head()

In [ ]:
pivot_train_df_csv_any = pivot_train_df_csv[pivot_train_df_csv['any'] == 1]
pivot_train_df_csv_nany = pivot_train_df_csv[pivot_train_df_csv['any'] != 1]

#pivot_train_df_csv_any = pivot_train_df_csv_any.iloc[1:50000]
pivot_train_df_csv_any = pivot_train_df_csv_any.sample(n=97103)
#pivot_train_df_csv_nany = pivot_train_df_csv_nany.iloc[1:50000]
pivot_train_df_csv_nany = pivot_train_df_csv_nany.sample(n=97103)

In [ ]:
pivot_train_df_csv=pd.concat([pivot_train_df_csv_any,pivot_train_df_csv_nany])

In [ ]:
pivot_train_df_csv=pivot_train_df_csv.reset_index(drop=True)

In [ ]:
pivot_train_df_csv.info()

In [ ]:
pivot_train_df_csv.head()

# Image Reading from disk path

# Read Test file

In [ ]:
# submission file is test file , need to conver into list of images
test_df_csv = pd.read_csv(BASE_PATH + 'stage_1_sample_submission.csv')

test_df_csv['filename'] = test_df_csv['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".dcm")
test_df_csv['type'] = test_df_csv['ID'].apply(lambda st: st.split('_')[2])

print(test_df_csv.shape)
test_df_csv.head()

In [ ]:
test_df_csv = pd.DataFrame(test_df_csv.filename.unique(), columns=['filename'])
print(test_df_csv.shape)
#test_df_csv.head()

# Data Generator Creation

In [ ]:
pivot_train_df_csv['filename'] = pivot_train_df_csv['filename'].apply(lambda st: st.split('.')[0] + ".png")
pivot_train_df_csv.head(2)

In [ ]:
test_df_csv['filename'] = test_df_csv['filename'].apply(lambda st: st.split('.')[0] + ".png")
test_df_csv.head(2)

np.set_printoptions(threshold=sys.maxsize)
imgpath = BASE_PATH + TEST_PATH+'ID_c16f5348b.png'

image = cv2.imread(imgpath,0)
image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,10), -4, 0)
plt.imshow(image)
print(image.shape)

IMG_SIZE = 224

image = cv2.imread(imgpath,0)
image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,10), -4, 0)
plt.imshow(image)

In [ ]:
print(BASE_PATH + TEST_PATH)
#reading all dcm files into train and text
# GLOB api work like unix style matching of path

train_dcm_img =  sorted(glob(BASE_PATH + TRAIN_PATH + "*.png"))
test_dcm_img  =  sorted(glob(BASE_PATH + TEST_PATH + "*.png"))

print("train files: ", len(train_dcm_img))
print("test files: ", len(test_dcm_img))

pd.reset_option('max_colwidth')

print(test_dcm_img[:4])
if '/kaggle/input/rsna_noben_png_224_224_v1/stage_1_test_images/ID_c16f5348b.png' in test_dcm_img:
    print('y')

In [ ]:
pivot_train_df_csv.head(5)

In [ ]:
num_ill_patients = pivot_train_df_csv[pivot_train_df_csv["any"]==1].shape[0]
num_ill_patients

In [ ]:
undersample_seed = 42
healthy_patients = pivot_train_df_csv[pivot_train_df_csv["any"]==0].index.values
healthy_patients_selection = np.random.RandomState(undersample_seed).choice(
    healthy_patients, size=num_ill_patients, replace=False
)
len(healthy_patients_selection)

In [ ]:
def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)
    """
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#   --- TO DO ----
#   zca_epsilon: epsilon for ZCA whitening. Default is 1e-6.
#   zca_whitening: Boolean. Apply ZCA whitening.
#
def create_datagen():
    train_aug = ImageDataGenerator(rescale=1./255, # Important as pixel haev 0 - 255 
                               horizontal_flip = True,
                               #zoom_range = 0.15,
                               #vertical_flip = True,
                               #shear_range=0.1,
                               rotation_range = 10,
                               validation_split=0.20,
                               preprocessing_function=preprocess_image
                               )
    return train_aug

In [ ]:
BATCH_SIZE = 32

def create_test_gen():
    return ImageDataGenerator(rescale=1./255,
                              preprocessing_function=preprocess_image).flow_from_dataframe(
        test_df_csv,
        directory= BASE_PATH+TEST_PATH,
        x_col='filename',
        class_mode=None,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=False
        
    )

In [ ]:
def create_flow(datagen, subset):
    return datagen.flow_from_dataframe(
        pivot_train_df_csv, 
        directory =BASE_PATH+TRAIN_PATH,
        x_col = 'filename', 
        y_col = ['any', 'epidural', 'intraparenchymal','intraventricular', 'subarachnoid', 'subdural'],
        class_mode='raw',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        subset=subset
    )

In [ ]:
%%time
# Using original generator
train_data_generator = create_datagen()

train_gen = create_flow(train_data_generator, 'training')
val_gen = create_flow(train_data_generator, 'validation')

test_gen = create_test_gen()

# Model Creation

In [ ]:
from keras.applications.xception import Xception
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import keras
from keras.engine import Layer,InputSpec
from keras.applications.resnet50 import ResNet50

In [ ]:
class RAdam(keras.optimizers.Optimizer):
    """RAdam optimizer.
    # Arguments
        learning_rate: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        learning_rate = kwargs.pop('lr', learning_rate)
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(learning_rate, name='learning_rate')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = K.maximum(self.total_steps - warmup_steps, 1)
            decay_rate = (self.min_lr - lr) / decay_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr + decay_rate * K.minimum(t - warmup_steps, decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t))
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t))

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t >= 5, r_t * m_corr_t / (v_corr_t + self.epsilon), m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    @property
    def lr(self):
        return self.learning_rate

    @lr.setter
    def lr(self, learning_rate):
        self.learning_rate = learning_rate

    def get_config(self):
        config = {
            'learning_rate': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# Loss Function

In [ ]:
#tf.losses.log_loss
import tensorflow as tf
from keras import backend as K

#weight_mask = [2.0,1.0,1.0,1.0,1.0,1.0]
#weights = tf.expand_dims(weight_mask, axis = 0)

def custom_log_loss(y_true, y_pred):
    loss_val = tf.losses.log_loss(y_true, y_pred)
    #loss_val = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    #loss_val = tf.losses.sigmoid_cross_entropy(y_true, y_pred)
    #print(y_true.shape)
    #print(ck_val)
    return loss_val

In [ ]:
#K.get_session().run(tf.local_variables_initializer())
#optimizer = RAdam(lr = 1e-4)
optimizer = keras.optimizers.adam(lr = .001,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0)

In [ ]:


class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
    

def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    

clr = CyclicLR(base_lr=0.001, max_lr=0.009,
                        step_size=300., mode='exp_range',
                        gamma=0.99994)

In [ ]:
input_layer = Input(shape = (IMG_SIZE,IMG_SIZE,3))
from keras.layers import BatchNormalization

base_model = ResNet50(weights = None,
                       include_top = False,
                       input_tensor = input_layer)

base_model.load_weights('../input/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
# all are false
#for layer in base_model.layers:
#    layer.trainable = False
# top 5 are fasle    
#for layer in base_model.layers[:180]:
#    layer.trainable = False
# last 4 are false
#for layer in vgg_conv.layers[:-4]:
#    layer.trainable = False
    
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
out = Dense(6, activation = 'sigmoid')(x)

model = Model(inputs = input_layer, outputs = out)
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint
#es = EarlyStopping(monitor='custom_log_Loss', mode='auto', verbose=1, patience=3,restore_best_weights=True)
#rlrop = ReduceLROnPlateau(monitor='custom_log_loss', 
#                        factor=0.2, 
#                        patience=5, 
#                        verbose=1, 
#                        mode='auto', 
#                        min_lr=1e-6)
filepath = "resNet50_DS_5epoch.h5"
mc=ModelCheckpoint(filepath, monitor='custom_log_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')


callback_list = [clr,mc]

model.compile(optimizer = optimizer, loss = custom_log_loss, metrics = ["accuracy",custom_log_loss]) 
model.summary()


In [ ]:
#print(675123 / BATCH_SIZE
print(len(train_gen))
print(len(train_gen)/BATCH_SIZE)
print(len(val_gen))

In [ ]:
class_weights = {0: 2.,
                1: 1.,
                2: 1.,
               3: 1.,
               4: 1.,
               5: 1.}

In [ ]:
%%time
#train_filenames = os.listdir(BASE_PATH + TRAIN_PATH)

#total_steps = len(train_filenames) / BATCH_SIZE

history = model.fit_generator(generator = train_gen, 
                    steps_per_epoch = len(train_gen), 
                    epochs = 5, 
                    validation_data = val_gen, 
                    validation_steps = len(val_gen), #total_steps * 0.15,
                    callbacks =  callback_list,
                    class_weight=class_weights)

In [ ]:
model.save_weights("Weight_resNet50_DS_5epoch_2.h5")
model.save('resNet50_DS_5epoch_2.h5')

Epoch 1/1
  839/17910 [>.............................] - ETA: 4:20:54 - loss: 0.1778 - accuracy: 0.6707 - custom_log_loss: 0.1778

In [ ]:
y_test = model.predict_generator(
    test_gen,
    steps=len(test_gen),
    verbose=1
)

In [ ]:
# Append the output predicts in the wide format to the y_test
test_df_csv = test_df_csv.join(pd.DataFrame(y_test, columns=[
    'any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural'
]))

# Unpivot table, i.e. wide (N x 6) to long format (6N x 1)
test_df_csv = test_df_csv.melt(id_vars=['filename'])

# Combine the filename column with the variable column
test_df_csv['ID'] = test_df_csv.filename.apply(lambda x: x.replace('.png', '')) + '_' + test_df_csv.variable
test_df_csv['Label'] = test_df_csv['value']

test_df_csv[['ID', 'Label']].to_csv('submission.csv', index=False)

In [ ]:
test_df_csv.head(20)